# Stack summarizer and downscaler

This script contains the code for:

* Summarizing the images from the given folder(s) into the float32 2-channel ImageJ-compatible tiff-image
* Creating the 2-channel Z-stack uint16 2-channel ImageJ-compatible tiff-image from the images in given folder(s) with it's simultaneous downscaling in X, Y and Z dimentions

*Also it has a part of examplary code for working in Google Collab on images from Google Drive.*

*Has examplary code for checking the number of images in input folder(s) and filtering based on it.*

In [ ]:
# Mounting the Google-drive, if it is used in Google Collab on images on Google disk
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing needed libraries
import glob
from PIL import Image
import numpy as np
from google.colab import files
import tifffile as tf
from tqdm import tqdm
from pathlib import Path
import imageio.v2 as imageio
import os
from skimage.transform import downscale_local_mean

### Define the functions for easy coding

In [ ]:
# If the images in the input folder are from the 2 channels, this function separate the given list of images to 2 separate lists based on 'ch1' and 'ch2' keywords presence in the each image name
def channel_splitter(folder):
  images_ch1 = []
  images_ch2 = []
  list_of_images = glob.glob(f'{folder}/*')
  for image in list_of_images:
    if "ch1" in image:
      images_ch1.append(image)
    elif 'ch2' in image:
      images_ch2.append(image)
  return images_ch1, images_ch2

In [ ]:
# This function uses as input the list of images and summarize them into 1 array of uint32 type
def summarizer(img_list):
  sum_array = imageio.imread(img_list[0]).astype(np.uint32)
  for image in img_list[1:]:
    image_array = imageio.imread(image).astype(np.uint32)
    sum_array += image_array
  return sum_array

In [ ]:
# This function uses as input the list of images, stack them in Z-dimantion and downscale in X Y and Z dimentions on the given factor (2, 2, 2) by default. May be changed when the function is called
def stack_downscaling(path_list, downscale_factor = (2, 2, 2)):
    first_image = imageio.imread(path_list[0])
    num_images = len(path_list)
    image_shape = first_image.shape
    image_tensors = np.empty((num_images,) + image_shape, dtype=first_image.dtype)
    image_tensors[0] = first_image
    for i, image_path in enumerate(path_list[1:]):
        image = imageio.imread(image_path)
        image_tensors[i + 1] = image
    image_tensors = downscale_local_mean(image_tensors, downscale_factor)
    return image_tensors

## Image processing step

In [ ]:
# This part of the code is used if multiple folders of images are given and checks if there are 1198 images: this folders are sent to a list for a summarizing task; if number of images in the folder is more than 1198, this folders are sent to another list for a downscaling task
to_summary_2channels = []
to_downscaling_z_stacks = []
for folder in glob.glob('/content/drive/MyDrive/2023_09_04/*/*/Exported Images'):
  if len(glob.glob(f'{folder}/*')) == 1198:
    to_summary_2channels.append(folder)
  elif len(glob.glob(f'{folder}/*')) > 1198:
    to_downscaling_z_stacks.append(folder)

### Summarizing the images of separate channels and saving the 2-channel tif

In [ ]:
# Iterating through a list of folders, processing and summarizing two channels of image data, and saving the result.
for folder in tqdm(to_summary_2channels):

# Define the output folder based on the channel content.
  output_folder = "/content/drive/MyDrive/Output"
  if 'hoechst' in folder:
    output_folder = output_folder + "/Hoechst"
  elif 'FOV2_Hoechst' in folder:
    output_folder = output_folder + "/FOV2_Hoechst"
  elif 'GFP' in folder:
    output_folder = output_folder + "/GFP"
# Create the output folder if it doesn't exist.
  path = Path(output_folder)
  path.mkdir(parents=True, exist_ok=True)
# Split the channels in the image folder by channel.
  images_ch1, images_ch2 = channel_splitter(folder)
# Summarize each channel.
  ch1 = summarizer(images_ch1)
  ch2 = summarizer(images_ch2)
# Combine the channels into a two-channel image and convert to the appropriate data type.
  two_channel_image = np.stack((ch1, ch2), axis=0)
  two_channel_image = two_channel_image.astype(np.float32)
# Save the two-channel image to a TIF file with a name based on the folder into the right output folder.
  tf.imwrite(f'{output_folder}/{folder.split("/")[-2]}.tif', two_channel_image, imagej=True)

### Creating the downscaled 2-channel Z-stacks

In [ ]:
# Iterate through a list of folders containing images for z-stack creating and downscaling
for folder in tqdm(to_downscaling_z_stacks):
# Define the output folder path based on the channel content.
  output_folder = "/content/drive/MyDrive/Output/Downscaled_zstacks"
  if 'hoechst' in folder:
    output_folder = output_folder + "/Hoechst"
  elif 'FOV2_Hoechst' in folder:
    output_folder = output_folder + "/FOV2_Hoechst"
  elif 'GFP' in folder:
    output_folder = output_folder + "/GFP"
# Create the output folder if it doesn't exist.
  path = Path(output_folder)
  path.mkdir(parents=True, exist_ok=True)
# Split the channels in the image folder by channel.
  ch1_list, ch2_list = channel_splitter(to_downscaling_z_stacks[0])
# Downscale and process the first channel and print processing status.
  ch1_array = stack_downscaling(ch1_list)
  print(folder.split("/")[-2])
  print('Ready channel 1; processed', len(ch1_list), 'images')
# Downscale and process the second channel and print processing status.
  ch2_array = stack_downscaling(ch2_list)
  print('Ready channel 2; processed', len(ch2_list), 'images')
 # Combine the processed channels into a two-channel image, and convert to uint16 data type.
  two_channel_image = np.stack((ch1_array, ch2_array), axis=1).astype(np.uint16)
# Save the two-channel image to a TIF file with a name based on the folder.
  tf.imwrite(f'{output_folder}/{folder.split("/")[-2]}.tif', two_channel_image, imagej=True)